In [91]:
import warnings
warnings.filterwarnings(action='ignore')

In [92]:
# Data Processing
import pandas as pd
import numpy as np
import requests

# Data Regru
import re

# Data Visualization
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
%matplotlib inline

# Machine Learning
from sklearn.metrics.pairwise import cosine_similarity 


# Purpose 
**Wine Recommendation based on your coffee taste**

- inspired by https://www.businessinsider.com/wine-reccomendations-based-on-coffee-2015-9

# Data crawling 
- data crawled from www.wine21.com
- price : under ₩ 200,000
- wine types limited to red, white, rose, sparkling and porto
- use only items being sold in off-line stores like E-mart, Lotte, Costco, and etc.

In [8]:
df_1 = pd.read_csv('./data/df_wine.csv')
df_1.drop(['Unnamed: 0'], axis=1, inplace=True)
df_1.tail(2)

,title,image_link,winery,country,province,grape_type,wine_type,occasion,alcohol,sweetness,acidity,body_rate,tannin_rate,food_pairing,price,vintage,size,link,aroma
4182,샤또 뽕따르니,http://img.wine21.com/WINE_MST/TITLE/0139000/W...,샤또 데스미레일,프랑스,보르도,"['카베르네 소비뇽', '메를로', '카베르네 프랑']",레드,테이블 와인,13.0,1,4,4,4,"그릴에 구운 비프스테이크 요리, 양고기 등과 잘 어울린다.",121000,2005,750,https://www.wine21.com/13_search/wine_view.htm...,"보랏빛 테두리를 가진 깊은 루비색을 띠며 바이올렛 등의 꽃향기와 베리, 체리류의 풍..."
4183,"라뮤라, 네로 다볼라",http://img.wine21.com/WINE_MST/TITLE/0139000/W...,라뮤라,이탈리아,시칠리아,['네로 다볼라'],레드,테이블 와인,8.5,1,4,3,3,"고기류, 파스타, 치즈와 향신료가 풍부한 음식들과 잘 어울린다.",19000,2002,750,https://www.wine21.com/13_search/wine_view.htm...,시칠리의 가장 중요한 포도품종인 네로 다볼라로 만들어졌으며 수작업으로 수확을 거친 ...


# Data Processing

In [9]:
df = df_1.copy()

## grape_type : use the first species

In [10]:
for i in range(len(df)):
    if ',' in df['grape_type'][i]:
        df['grape_type'][i] = df['grape_type'][i][2:-2].split(',')[0][:-1]
    else:
        df['grape_type'][i] = df['grape_type'][i][2:-2]
df['grape_type']

0           바르베라
1           바르베라
2           샤르도네
3          피노 누아
4          피노 누아
5           샤르도네
6       카베르네 소비뇽
7           샤르도네
8            메를로
9           샤르도네
10          샤르도네
11          샤르도네
12          샤르도네
13           메를로
14          마르산느
15      카베르네 소비뇽
16         산지오베제
17          비오시뉴
18      투리가 프란체사
19        시라/쉬라즈
20            말벡
21         슈냉 블랑
22           메를로
23          샤르도네
24          샤르도네
25      카베르네 소비뇽
26           진판델
27         모나스트렐
28        소비뇽 블랑
29         모나스트렐
          ...   
4154       산지오베제
4155       산지오베제
4156       산지오베제
4157    프루놀로 젠타일
4158    프루놀로 젠타일
4159       산지오베제
4160       산지오베제
4161         메를로
4162       산지오베제
4163    카베르네 소비뇽
4164         메를로
4165        네비올로
4166       산지오베제
4167       산지오베제
4168       산지오베제
4169         메를로
4170        샤르도네
4171      시라/쉬라즈
4172        샤르도네
4173         메를로
4174    카베르네 소비뇽
4175      시라/쉬라즈
4176      소비뇽 블랑
4177       피노 누아
4178        샤르도네
4179         메를로
4180       산지오베제
4181    카베르네 소

In [11]:
df['grape_type'].unique()

array(['바르베라', '샤르도네', '피노 누아', '카베르네 소비뇽', '메를로', '마르산느', '산지오베제',
       '비오시뉴', '투리가 프란체사', '시라/쉬라즈', '말벡', '슈냉 블랑', '진판델', '모나스트렐',
       '소비뇽 블랑', '그르나슈', '쌩쏘', '모스카토', '블랜드', '템프라니요', '네로 다볼라', '프리미티보',
       '카르메네르', '투리가 나시오날', '자렐로', '베르데호', '아시르티코', '시노마브로', '마브루드',
       '아기오르기티코', '로디티스', '빌라나', '까리냥', '모스카텔 세투발', '가르나차', '피아노', '마카베오',
       '틴타 네그라 몰레', '아이렌', '리슬링', '글레라', '오미자', '코르비나 베로네제', '쁘띠 시라',
       '소비뇽 그리', '베르디키오 비앙코', '몬테풀치아노', '라크리마', '비오니에', '푸르민트', '젤렌',
       '트라하두라', '멘시아', '보발', '돌체토', '코르테제', '아르네이스', '네비올로', '뮈스까 까넬리',
       '세르시알', '알리아니코', '베르멘티노', '카베르네 프랑', '믈롱 드 부르고뉴', '세미용', '피노 네로',
       '피노 그리지오', '피노 뮈니에', '가르나차 틴토레라', '그르나슈 블랑', '레불라', '쁘띠 메슬리에',
       '모작', '네로 디 트로이아', '레포스코', '말바시아 디 칸디다', '클레렛', '피노 그리',
       '페르나오 피레스', '아라고네즈', '코르비나', '론디넬라', '롤', '카스텔라옹', '비칼', '사그란티노',
       '고데요', '트레비아노', '알바리뇨', '플라바치 말리', '포십', '게뷔르츠트라미너',
       '모스까뗄 데 알레한드리아', '오르테가', '모스카텔', '틴타 로리즈', '청포도 즙', '나스세따',
       '쁘띠 쉬라', '피에디로쏘', '그

In [12]:
# dropped 'wine' made from apple
df.drop(index=df[df['grape_type']=='사과'].index, inplace=True)
df.reset_index(inplace=True)

## one_hot encoding

### food_pairing : one_hot encoding
- add meat / cheese / seafood / vegetables / dessert columns
- meat : 고기, 육류, 돼지, 소, 비프,  닭, 오리, 양, 스테이크, 보쌈, 냉채, 바비큐, 하몽, 불고기, 햄, 쇠고기, 소시지, 칠면조, 치킨,
- cheese : 치즈
- seafood : 해산물, 조개, 새우, 갑각류, 연어, 참치, 초밥, 생선, 굴, 회, 사시미, 관자,
- vegetables : 버섯, 딸기, 올리브, 아몬드, 샐러드, 아보카도, 과일, 호두, 멜론, 무화과, 야채, 견과류, 
- dessert : 디저트, 수플레, 케익, 케이크, 파이, 마카롱, 아이스크림,

In [13]:
meat_ls = ['고기', '육류', '돼지', '소', '비프', '닭', '오리', '양', '스테이크', '보쌈', '냉채', '바비큐', 
           '하몽', '불고기', '햄', '쇠고기', '소시지', '칠면조', '치킨']
cheese_ls = ['치즈']
seafood_ls = ['해산물', '조개', '새우', '갑각류', '연어', '참치', '초밥', '생선', '굴', '회', '사시미', '관자']
vegetables_ls = ['버섯', '딸기', '올리브', '아몬드', '샐러드', '아보카도', '과일', '호두', '멜론', '무화과', '야채', '견과류']
dessert_ls = ['디저트', '수플레', '케익', '케이크', '파이', '마카롱', '아이스크림']

In [14]:
df['food_pairing']

0       피자, 파스타,  붉은 육류,  그리고 치즈 플레이트와 잘 어울리며, 특유의 산도가...
1                    식전 와인으로 적합하며 전체요리나 기름진 생선요리와 잘 어울린다.
2                                 치즈, 해산물, 샐러드 등과 잘 어울린다.
3                                     치즈, 오리고기 등과 잘 어울린다.
4                        치즈, BBQ, 구운 고기, 버섯 요리 등과 잘 어울린다.
5                         해산물 샐러드, 생선요리, 가금류요리 등과 잘 어울린다.
6                        향이 강한 치즈, 구운 고기류, 파스타 등과 잘 어울린다.
7                   생선 요리, 조개류, 흰 살 고기를 활용한 요리 등과 잘 어울린다.
8                                     치즈, 스테이크 등과 잘 어울린다.
9                        해산물 플래터, 바다가재, 가자미 요리 등과 잘 어울린다.
10                 돼지고기, 연어, 참치, 갑각류, 향이 연한 치즈 등과 잘 어울린다.
11         연어, 참치, 갑각류, 향이 연하고 부드러운 치즈, 치킨 파히타 등과 잘 어울린다.
12               해산물 요리, 훈제 연어, 초밥, 새위튀김, 브리치즈 등과 잘 어울린다.
13           모든 육류, 치즈와 잘 어울림, 연어와 버섯을 곁들인 파스타 등과 잘 어울린다.
14                              바다가재, 생선류, 해산물 등과 잘 어울린다.
15                           스테이크, 까망베르치즈, 초콜렛 등과 잘 어울린다.
16                         훈제, 양념 치즈, 미트소스 파스타 등과 잘 어울린다.
17            

In [15]:
# add meat category column
df['meat'] = 0
df['cheese'] = 0
df['seafood'] = 0
df['vegetables'] = 0
df['dessert'] = 0

for i in range(len(df['food_pairing'])):
    for meat in meat_ls:
        if meat in df['food_pairing'][i]:
            df['meat'][i] = 1
    for cheese in cheese_ls:
        if cheese in df['food_pairing'][i]:
            df['cheese'][i] = 1
    for seafood in seafood_ls:
        if seafood in df['food_pairing'][i]:
            df['seafood'][i] = 1
    for vegetables in vegetables_ls:
        if vegetables in df['food_pairing'][i]:
            df['vegetables'][i] = 1
    for dessert in dessert_ls:
        if dessert in df['food_pairing'][i]:
            df['dessert'][i] = 1

In [16]:
df[['food_pairing', 'meat', 'cheese', 'seafood', 'vegetables', 'dessert']]

,food_pairing,meat,cheese,seafood,vegetables,dessert
0,"피자, 파스타, 붉은 육류, 그리고 치즈 플레이트와 잘 어울리며, 특유의 산도가...",1,1,0,0,0
1,식전 와인으로 적합하며 전체요리나 기름진 생선요리와 잘 어울린다.,0,0,1,0,0
2,"치즈, 해산물, 샐러드 등과 잘 어울린다.",0,1,1,1,0
3,"치즈, 오리고기 등과 잘 어울린다.",1,1,0,0,0
4,"치즈, BBQ, 구운 고기, 버섯 요리 등과 잘 어울린다.",1,1,0,1,0
5,"해산물 샐러드, 생선요리, 가금류요리 등과 잘 어울린다.",0,0,1,1,0
6,"향이 강한 치즈, 구운 고기류, 파스타 등과 잘 어울린다.",1,1,0,0,0
7,"생선 요리, 조개류, 흰 살 고기를 활용한 요리 등과 잘 어울린다.",1,0,1,0,0
8,"치즈, 스테이크 등과 잘 어울린다.",1,1,0,0,0
9,"해산물 플래터, 바다가재, 가자미 요리 등과 잘 어울린다.",0,0,1,0,0


### aroma categories
- append aroma category dummies based on string values in aroma_y column
- floral / fruit / cologne / oriental_leather / oriental_spice / earth*
* aroma criteria based off of wine aroma wheel from
https://www.kellyandjones.com/blogs/news/perfumers-guide-to-wine-aromas

In [17]:
floral = ['로럴', '백합', '자스민', '제라니움', '쟈스민', '피오니', '꽃', '아이리스', '부케', '아카시아', '장미', '클로버', '로즈힙',
         '플라워', '카모마일']
fruit = ['트로피칼', '복숭아', '과일','과육', '열매', '스트로베리', '딸기', '블루베리', '베리', '아로니아', '과실', '라스베리', '자두', '체리', 
         '카시스', '사과', '배향', '커런트', '멜론', '애플', '프룻', '매실', '프루티', '살구', '애플리콧', '배', '무화과', '플럼', 
         '패션후르츠', '오디', '모과', '파파야', '망고', '석류', '버찌', '코코넛', '바나나', '호박']
citrus = ['시트러스', '씨트런향', '자몽', '오렌지', '레몬', '귤', '라임', '탠저린', '베르가못']

oriental_leather = ['에스프레소', '담배', '가죽', '쵸콜릿', '바닐라', '카라멜', '초콜릿', '캬라멜', '스모크', '흑연', '모카', 
                    '스파이', '꿀',  '헤이즐넛', '코코아', '타바코', '시가', '레더', '아몬드', '호두','허니', '크림']
oriental_spice = ['향신료', '세이지', '감초', '씨나몬', '라벤다', '라벤더', '발사믹','스파이스' '스파이시', '계피', '정향', '페퍼', 
                  '후추']
earth = ['낙엽', '흙내', '커피', '삼나무', '로즈마리', '나무', '버섯', '허브', '유칼립투스', '토양', '밀짚','회향', '젖은 돌', 
         '머쉬룸', '트러플']


In [21]:
# convert data type from object to string type data

df.aroma = df.aroma.apply(lambda x: str(x))

In [22]:
# add floral category column

df['floral'] = 0
df['fruit'] = 0
df['citrus'] = 0
df['oriental_spice'] = 0
df['oriental_leather'] = 0
df['earth'] = 0

for i in range(len(df)):
    for floral_ in floral:
        if floral_ in df['aroma'][i]:
            df['floral'][i] = 1
    for fruit_ in fruit:
        if fruit_ in df['aroma'][i]:
            df['fruit'][i] = 1
    for citrus_ in citrus:
        if citrus_ in df['aroma'][i]:
            df['citrus'][i] = 1
    for oriental_spice_ in oriental_spice:
        if oriental_spice_ in df['aroma'][i]:
            df['oriental_spice'][i] = 1
    for oriental_leather_ in oriental_leather:
        if oriental_leather_ in df['aroma'][i]:
            df['oriental_leather'][i] = 1
    for earth_ in earth:
        if earth_ in df['aroma'][i]:
            df['earth'][i] = 1

In [23]:
for name in ['floral', 'fruit', 'citrus', 'oriental_spice', 'oriental_leather','earth']:
    print(df[name].value_counts())

0    3336
1     847
Name: floral, dtype: int64
1    3499
0     684
Name: fruit, dtype: int64
0    3538
1     645
Name: citrus, dtype: int64
0    3418
1     765
Name: oriental_spice, dtype: int64
0    2604
1    1579
Name: oriental_leather, dtype: int64
0    3435
1     748
Name: earth, dtype: int64


#### dark/fruity aroma
- aroma category combinations created for filtering wine recommendation list based on coffee type preference

In [24]:
# fruity aroma : floral, fruit, citrus

df[((df['floral'] ==1) | (df['fruit'] == 1) | (df['citrus'] == 1)) & (df['oriental_spice'] ==0) & 
   (df['oriental_leather'] ==0) & (df['earth'] ==0)]

,index,title,image_link,winery,country,province,grape_type,wine_type,occasion,alcohol,...,cheese,seafood,vegetables,dessert,floral,fruit,citrus,oriental_spice,oriental_leather,earth
0,0,트리풀라 피에몬테 로쏘,http://img.wine21.com/WINE_MST/TITLE/0167000/W...,몬도 델 비노,이탈리아,피에몬테,바르베라,레드,테이블 와인,13.5,...,1,0,0,0,1,1,0,0,0,0
1,1,아케시 로사토 브뤼,http://img.wine21.com/WINE_MST/TITLE/0167000/W...,몬도 델 비노,이탈리아,피에몬테,바르베라,스파클링,테이블 와인,11.5,...,0,1,0,0,1,1,0,0,0,0
2,2,엘리자베스 로제 샤도네이,http://img.wine21.com/WINE_MST/TITLE/0167000/W...,고스트 블락 와이너리,미국,캘리포니아,샤르도네,화이트,테이블 와인,13.5,...,1,1,1,0,0,1,1,0,0,0
5,5,"백하우스, 샤도네이",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,백하우스,미국,캘리포니아,샤르도네,화이트,테이블 와인,13.5,...,0,1,1,0,0,1,1,0,0,0
7,7,"올리비에 르플레브, 부르고뉴 “옹끌 뱅썽”",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,올리비에 르플레브,프랑스,부르고뉴,샤르도네,화이트,테이블 와인,13.5,...,0,1,0,0,0,1,1,0,0,0
10,10,"도멘 앙리, 샤블리 프리미에 크뤼 ‘트호엠므’",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,도멘 앙리,프랑스,부르고뉴,샤르도네,화이트,테이블 와인,12.5,...,1,1,0,0,1,1,1,0,0,0
11,11,"도멘 앙리, 샤블리 ‘쌩 삐에흐’",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,도멘 앙리,프랑스,부르고뉴,샤르도네,화이트,테이블 와인,13.5,...,1,1,0,0,1,1,1,0,0,0
12,12,"도멘 앙리, 821 블랑 드 블랑 브뤼",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,도멘 앙리,프랑스,부르고뉴,샤르도네,스파클링,테이블 와인,12.5,...,1,1,0,0,0,1,1,0,0,0
13,13,샤또 라 로즈 몽비엘,http://img.wine21.com/WINE_MST/TITLE/0167000/W...,비뇨블 페레베쥬,프랑스,보르도,메를로,레드,테이블 와인,13.5,...,1,1,1,0,0,1,0,0,0,0
14,14,"레어 빈야드, 마르산느 비오니에",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,레어 빈야드,프랑스,서던,마르산느,화이트,테이블 와인,12.5,...,0,1,0,0,0,1,0,0,0,0


In [25]:
# dark aroma : oriental_spice, oriental_leather, earth

df[((df['oriental_spice'] == 1) | (df['oriental_leather'] == 1) | (df['earth'] ==1)) & 
  (df['floral'] == 0) & (df['fruit'] == 0) & (df['citrus'] == 0)]

,index,title,image_link,winery,country,province,grape_type,wine_type,occasion,alcohol,...,cheese,seafood,vegetables,dessert,floral,fruit,citrus,oriental_spice,oriental_leather,earth
136,136,"아나이, 파운더스 콜렉션 시라",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,아나이,칠레,라펠,시라/쉬라즈,레드,테이블 와인,13.5,...,0,0,0,0,0,0,0,1,0,0
146,146,리저브 생 마르땡 까베르네 소비뇽,http://img.wine21.com/WINE_MST/TITLE/0167000/W...,리저브 생 마르땡,프랑스,서던,카베르네 소비뇽,레드,테이블 와인,8.5,...,0,0,0,0,0,0,0,1,0,1
287,287,"에구렌 우가르떼, 크리안자",http://img.wine21.com/WINE_MST/TITLE/0166000/W...,에구렌 우가르떼,스페인,리오하,템프라니요,레드,테이블 와인,13.5,...,0,1,0,0,0,0,0,0,1,1
288,288,"에구렌 우가르떼, 코세차",http://img.wine21.com/WINE_MST/TITLE/0166000/W...,에구렌 우가르떼,스페인,리오하,템프라니요,레드,테이블 와인,13.5,...,0,1,0,0,0,0,0,0,1,1
289,289,"에구렌 우가르떼, 레세르바",http://img.wine21.com/WINE_MST/TITLE/0166000/W...,에구렌 우가르떼,스페인,리오하,템프라니요,레드,테이블 와인,14.5,...,0,1,0,0,0,0,0,0,1,1
312,312,"리버비, 싱글 빈야드 샤르도네",http://img.wine21.com/WINE_MST/TITLE/0166000/W...,리버비 에스테이트,뉴질랜드,사우스,샤르도네,화이트,테이블 와인,12.5,...,0,0,0,0,0,0,0,0,0,1
555,555,"도멘 드 라 호예르, 루베롱 블랑",http://img.wine21.com/WINE_MST/TITLE/0164000/W...,도멘 드 라 호예르,프랑스,론,그르나슈 블랑,화이트,테이블 와인,13.5,...,1,1,1,0,0,0,0,0,1,1
648,648,오르넬로,http://img.wine21.com/WINE_MST/TITLE/0163000/W...,까스텔라레 디 까스텔리나,이탈리아,토스카나,산지오베제,레드,테이블 와인,14.5,...,1,0,0,0,0,0,0,0,1,0
674,674,"테사리, 아브스",http://img.wine21.com/WINE_MST/TITLE/0163000/0...,테사리,이탈리아,베네토,가르가네가,화이트,테이블 와인,12.5,...,0,1,0,0,0,0,0,0,1,1
789,789,"샤또 수쉐리, 꼬또 뒤 레이용 비에이 비뉴",http://img.wine21.com/WINE_MST/TITLE/0162000/W...,샤또 수쉐리,프랑스,루아르,슈냉 블랑,화이트,테이블 와인,11.5,...,1,1,0,0,0,0,0,0,0,1


In [26]:
df['dark_aroma'] = 0
df['fruity_aroma'] = 0

for i in df[(df.oriental_spice==1) | (df.oriental_leather==1) | (df.earth==1)].index:
    df['dark_aroma'][i] = 1

for i in df[(df.floral==1) | (df.fruit==1) | (df.citrus==1)].index:
    df['fruity_aroma'][i] = 1

In [27]:
df[['dark_aroma', 'fruity_aroma']]

,dark_aroma,fruity_aroma
0,0,1
1,0,1
2,0,1
3,1,1
4,1,1
5,0,1
6,1,1
7,0,1
8,1,1
9,1,1


In [28]:
# drop wines which weren't categorized by aroma
print('wines without aroma category : ', len(df[(df['dark_aroma'] == 0) & (df['fruity_aroma'] == 0)]))

df = df[(df['dark_aroma'] == 1) | (df['fruity_aroma'] == 1)]
df.reset_index(inplace=True)

print('data count : ', len(df))

wines without aroma category :  470
data count :  3713


### continent
- grouping countries based on their geographic location and climate
- necessary to reduce bias due to overfitting

- Europe_A : 프랑스, 오스트리아, 몰도바, 독일, 헝가리, 슬로베니아, 불가리아, 조지아, 루마니아
- Europe_B : 이탈리아, 포르투갈, 스페인, 그리스, 크로아티아, 이스라엘, 레바논
- North_America : 미국, 캐나다
- South_America : 아르헨티나, 칠레, 우루과이
- New_World : 호주, 뉴질랜드, 남아프리카
- Korea : 대한민국

In [29]:
europe_a = ['프랑스', '오스트리아', '몰도바', '독일', '헝가리', '슬로베니아', '불가리아', '조지아', '루마니아']
europe_b = ['이탈리아', '포르투갈', '스페인', '그리스', '크로아티아', '이스라엘', '레바논']
north_america = ['미국', '캐나다']
south_america = ['아르헨티나', '칠레', '우루과이'] 
new_world = ['호주', '뉴질랜드', '남아프리카']
korea = ['대한민국']

In [30]:
df['europe_a'] = 0
df['europe_b'] = 0
df['north_america'] = 0
df['south_america'] = 0
df['new_world'] = 0
df['korea'] = 0

for i in range(len(df['country'])):
    for a in europe_a:
        if a in df['country'][i]:
            df['europe_a'][i] = 1
    for b in europe_b:
        if b in df['country'][i]:
            df['europe_b'][i] = 1
    for north in north_america:
        if north in df['country'][i]:
            df['north_america'][i] = 1
    for south in south_america:
        if south in df['country'][i]:
            df['south_america'][i] = 1
    for new in new_world:
        if new in df['country'][i]:
            df['new_world'][i] = 1
    for k in korea:
        if k in df['country'][i]:
            df['korea'][i] = 1
            
df[['country', 'europe_a', 'europe_b', 'north_america', 'south_america', 'new_world', 'korea']]

,country,europe_a,europe_b,north_america,south_america,new_world,korea
0,이탈리아,0,1,0,0,0,0
1,이탈리아,0,1,0,0,0,0
2,미국,0,0,1,0,0,0
3,미국,0,0,1,0,0,0
4,미국,0,0,1,0,0,0
5,미국,0,0,1,0,0,0
6,미국,0,0,1,0,0,0
7,프랑스,1,0,0,0,0,0
8,프랑스,1,0,0,0,0,0
9,프랑스,1,0,0,0,0,0


In [31]:
# count european wines
print('europe_a wine_count : ', len(df[(df['europe_a'] == 1)]))
print('europe_b wine_count : ', len(df[(df['europe_b'] == 1)]))
print('north_america wine_count : ', len(df[(df['north_america'] == 1)]))
print('south_america wine_count : ', len(df[(df['south_america'] == 1)]))
print('new_world wine_count : ', len(df[(df['new_world'] == 1)]))
print('korea wine_count : ', len(df[(df['korea'] == 1)]))

europe_a wine_count :  974
europe_b wine_count :  1205
north_america wine_count :  439
south_america wine_count :  676
new_world wine_count :  408
korea wine_count :  11


## wine_type 
- '로제' & '스파클링' -> '화이트'
- '주정강화' -> '레드' or '화이트'
    - port wine -> '레드'
    - the others -> '삭제'

In [32]:
df['wine_type'].unique()

array(['레드', '스파클링', '화이트', '주정강화', '로제'], dtype=object)

In [33]:
df[(df['wine_type'] == '로제') | (df['wine_type'] == '스파클링')]['wine_type'] = '화이트'

In [34]:
for i in range(len(df)):
    if '포트' in df['title'][i]:
        df['wine_type'][i] = '레드'

In [35]:
print("Red Counts : " , round((len(df[df['wine_type'] == '레드']) / len(df)) * 100, 2), "%")
print("White Counts : " , round((len(df[df['wine_type'] == '화이트']) / len(df)) * 100, 2), "%")

Red Counts :  63.37 %
White Counts :  26.07 %


In [36]:
for i in range(len(df)):
    if df['wine_type'][i] == '레드':
        df['wine_type'][i] = 'red'
    elif df['wine_type'][i] == '화이트':
        df['wine_type'][i] = 'white'
    else:
        df['wine_type'][i] = 'etc'

In [37]:
# delect etc(sherry wine)
for i in range(len(df)):
    if df['wine_type'][i] == 'etc':
        df2 = df.drop([i], inplace=True)

In [38]:
len(df)

3321

In [44]:
df['red'] = 0
df['white'] = 0

In [45]:
# one_hot encdoing for wine_type
df[['red', 'white']] = pd.get_dummies(df['wine_type'])

In [46]:
print("Red Counts : " , round((len(df[df['wine_type'] == 'red']) / len(df)) * 100, 2), "%")
print("White Counts : " , round((len(df[df['wine_type'] == 'white']) / len(df)) * 100, 2), "%")

Red Counts :  70.85 %
White Counts :  29.15 %


In [47]:
df.columns

Index(['level_0', 'index', 'title', 'image_link', 'winery', 'country',
       'province', 'grape_type', 'wine_type', 'occasion', 'alcohol',
       'sweetness', 'acidity', 'body_rate', 'tannin_rate', 'food_pairing',
       'price', 'vintage', 'size', 'link', 'aroma', 'meat', 'cheese',
       'seafood', 'vegetables', 'dessert', 'floral', 'fruit', 'citrus',
       'oriental_spice', 'oriental_leather', 'earth', 'dark_aroma',
       'fruity_aroma', 'europe_a', 'europe_b', 'north_america',
       'south_america', 'new_world', 'korea', 'red', 'white'],
      dtype='object')

## data reduction

In [48]:
# drop unnessary columns
df.drop('occasion', axis=1, inplace=True)
df.drop('index', axis=1, inplace=True)
df.drop('level_0', axis=1, inplace=True)

In [49]:
# delete tesco wines
df[(df['winery'] == '테스코 파이니스트') | (df['winery'] == '테스코 그룹  (테스코 심플리 와인) ')] = \
df[(df['winery'] == '테스코 파이니스트') | (df['winery'] == '테스코 그룹  (테스코 심플리 와인) ')].drop(index=df[(df['winery'] == '테스코 파이니스트') | (df['winery'] == '테스코 그룹  (테스코 심플리 와인) ')].index
, inplace=True)

In [50]:
# delete wine's size either over or below than 750ml
df = df[df['size'] == 750.0]

In [51]:
# delete image(NAN)

df = df[df.image_link.notnull()]

In [52]:
df.reset_index(drop=True, inplace=True)
df['wine_id'] = df.index

In [53]:
df.isnull().sum()

title                 0
image_link            0
winery                0
country               0
province            181
grape_type            0
wine_type             0
alcohol               0
sweetness             0
acidity               0
body_rate             0
tannin_rate           0
food_pairing          0
price                 0
vintage              29
size                  0
link                  0
aroma                 0
meat                  0
cheese                0
seafood               0
vegetables            0
dessert               0
floral                0
fruit                 0
citrus                0
oriental_spice        0
oriental_leather      0
earth                 0
dark_aroma            0
fruity_aroma          0
europe_a              0
europe_b              0
north_america         0
south_america         0
new_world             0
korea                 0
red                   0
white                 0
wine_id               0
dtype: int64

In [55]:
df.to_csv('./data/df_aroma_processing.csv', encoding='utf-8-sig')

In [80]:
df = pd.read_csv('./data/df_aroma_processing.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,title,image_link,winery,country,province,grape_type,wine_type,alcohol,sweetness,acidity,...,fruity_aroma,europe_a,europe_b,north_america,south_america,new_world,korea,red,white,wine_id
0,트리풀라 피에몬테 로쏘,http://img.wine21.com/WINE_MST/TITLE/0167000/W...,몬도 델 비노,이탈리아,피에몬테,바르베라,red,13.5,1.0,3.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
1,엘리자베스 로제 샤도네이,http://img.wine21.com/WINE_MST/TITLE/0167000/W...,고스트 블락 와이너리,미국,캘리포니아,샤르도네,white,13.5,1.0,4.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1
2,"맥매니스, 캘리포니아 피노 누아",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,맥매니스 와이너리,미국,캘리포니아,피노 누아,red,13.5,1.0,4.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2
3,"백하우스, 피노 누아",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,백하우스,미국,캘리포니아,피노 누아,red,13.5,1.0,4.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3
4,"백하우스, 샤도네이",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,백하우스,미국,캘리포니아,샤르도네,white,13.5,1.0,3.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,4
5,"백하우스, 까베르네 소비뇽",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,백하우스,미국,캘리포니아,카베르네 소비뇽,red,13.5,1.0,3.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,5
6,"올리비에 르플레브, 부르고뉴 “옹끌 뱅썽”",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,올리비에 르플레브,프랑스,부르고뉴,샤르도네,white,13.5,1.0,4.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6
7,레이디 그레이삭,http://img.wine21.com/WINE_MST/TITLE/0167000/W...,도멘 롤랑드비,프랑스,보르도,메를로,red,13.5,1.0,3.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7
8,"도멘 앙리, 샤블리 프리미에 크뤼 ‘푸흐숌므’",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,도멘 앙리,프랑스,부르고뉴,샤르도네,white,13.5,1.0,4.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8
9,"도멘 앙리, 샤블리 프리미에 크뤼 ‘트호엠므’",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,도멘 앙리,프랑스,부르고뉴,샤르도네,white,12.5,1.0,4.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9


# Questionnaire for service user

###### Survey

> 1) Which coffee do you prefer to drink?
1. black coffee (e.g. americano, espresso)
2. coffee with milk & sugar (eg. frappuccino, vanila latte, etc.)
3. I prefer tea

> 2) Which type of coffee bean (or tea) do you prefer?
1. acidic type
2. full body type

###### cases (1000 cases per type)
- dark flavor : Earth, Oriental / fruity flavor : floral, fruity, cologne
1. type A : black - acidic : Dry / light / high acidity / dark flavors
2. type B : black - bodied : Dry / bold / low acidity / dark flavors
3. type C : cafe latte with sugar - acidic : sweet / bold / high acidity 
4. type D : cafe latte with sugar - bodied : sweet / bold / low acidity 
5. type E : Tea - acidic : Dry / light / high acidity / fruity flavors 
6. type F : Tea - bodied : Dry / bold / low acidity / fruity flavors

## Data Clustering

> 1) Which coffee do you prefer to drink?
- if black or tea: sweetness below 3 -> type a, b, e or f
- if cafe latte w/ sugar: sweetness greater or equal to 3 -> type c or d

> 2) Which type of coffee bean (or tea) do you prefer?
- acidic type: body rate below 3 & acidity greater than 3 -> type a or e
- acidic type: body rate greater than 3 & acidity equal to or lower than 3 -> type c
- full body type: body rate greater than 3 * acidity equal to or lower than 3 -> type b, d or f
    
* dark aroma checked -> type a or b
* fruity aroma checked -> type e or f

### wine_data 

In [57]:
# need more wine data for type_c & d

type_a = df[(df['sweetness'] < 2) & (df['body_rate'] < 3) & (df['acidity'] > 3) & (df['dark_aroma'] == 1)]
type_b = df[(df['sweetness'] < 2) & (df['body_rate'] > 2) & (df['acidity'] <= 3) & (df['dark_aroma'] == 1)]
type_c = df[(df['sweetness'] > 1) & (df['body_rate'] > 3) & (df['acidity'] > 3)]
type_d = df[(df['sweetness'] > 1) & (df['body_rate'] > 3) & (df['acidity'] <= 3)]
type_e = df[(df['sweetness'] < 2) & (df['body_rate'] < 3) & (df['acidity'] > 3) & (df['fruity_aroma'] == 1)]
type_f = df[(df['sweetness'] < 2) & (df['body_rate'] > 2) & (df['acidity'] <= 3) & (df['fruity_aroma'] == 1)]

# add type column
type_a['type'] = 'a'
type_b['type'] = 'b'
type_c['type'] = 'c'
type_d['type'] = 'd'
type_e['type'] = 'e'
type_f['type'] = 'f'

# concat all types into contegrated dataframe
wine_data = pd.concat([type_a, type_b,type_c,type_d,type_e,type_f])
wine_data.tail(2)

,title,image_link,winery,country,province,grape_type,wine_type,alcohol,sweetness,acidity,...,europe_a,europe_b,north_america,south_america,new_world,korea,red,white,wine_id,type
3175,마르체시 토리지아니 또레 디 치아르도,http://img.wine21.com/WINE_MST/TITLE/0139000/W...,마르체시 토리지아니,이탈리아,토스카나,산지오베제,red,9.0,1.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,3175,f
3176,샤또 몽그랑 밀론,http://img.wine21.com/WINE_MST/TITLE/0139000/W...,루시앙 루똥,프랑스,보르도,카베르네 소비뇽,red,13.0,1.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3176,f


In [82]:
wine_data.to_csv('./data/wine_data.csv', encoding='utf-8-sig')

In [67]:
print('type_a count :', len(wine_data[wine_data['type']== 'a']))
print('type_b count :', len(wine_data[wine_data['type']== 'b']))
print('type_c count :', len(wine_data[wine_data['type']== 'c']))
print('type_d count :', len(wine_data[wine_data['type']== 'd']))
print('type_e count :', len(wine_data[wine_data['type']== 'e']))
print('type_f count :', len(wine_data[wine_data['type']== 'f']))

type_a count : 90
type_b count : 1163
type_c count : 18
type_d count : 53
type_e count : 223
type_f count : 1637


### customer_data

In [68]:
customer_data = pd.DataFrame(columns=['title', 'image_link', 'winery', 'country', 'province',
       'grape_type', 'wine_type', 'alcohol', 'sweetness', 'acidity',
       'body_rate', 'tannin_rate', 'food_pairing', 'price', 'vintage', 'size',
       'link', 'aroma_y', 'meat', 'cheese', 'seafood', 'vegetables', 'dessert',
       'floral', 'fruit', 'citrus', 'oriental_spice', 'oriental_leather',
       'earth', 'dark_aroma', 'fruity_aroma', 'europe_a', 'europe_b',
       'north_america', 'south_america', 'new_world', 'korea', 'red', 'white',
       'wine_id','type'])

# type_a customers
for _ in range(2000):
    customer_data = customer_data.append(wine_data[wine_data['type']=='a'].sample(3), ignore_index=True)
    
# type_b customers
for _ in range(2000):
    customer_data = customer_data.append(wine_data[wine_data['type']=='b'].sample(3), ignore_index=True)

# type_c customers
for _ in range(2000):
    customer_data = customer_data.append(wine_data[wine_data['type']=='c'].sample(3), ignore_index=True)

# type_d customers
for _ in range(2000):
    customer_data = customer_data.append(wine_data[wine_data['type']=='d'].sample(3), ignore_index=True)

# type_e customers
for _ in range(2000):
    customer_data = customer_data.append(wine_data[wine_data['type']=='e'].sample(3), ignore_index=True)

# type_f customers
for _ in range(1000):
    customer_data = customer_data.append(wine_data[wine_data['type']=='f'].sample(3), ignore_index=True)

In [69]:
customer_data.tail(2)

,acidity,alcohol,aroma,aroma_y,body_rate,cheese,citrus,country,dark_aroma,dessert,...,sweetness,tannin_rate,title,type,vegetables,vintage,white,wine_id,wine_type,winery
32998,3.0,14.5,"강렬한 보라빛이 감도는 레드 컬러로 달콤한 블랙 베리, 감초, 허브와 가죽향이 지배...",NaN,4.0,0.0,0.0,프랑스,1.0,0.0,...,1.0,4.0,"뛰느방 깔베, 뀌베 꽁스땅스",f,0.0,2018,0.0,1321,red,뛰느방 깔베
32999,3.0,9.0,설탕 절임을 한듯한 붉은 과일의 달콤한 아로마가 진하게 풍겨오는 와인으로 입에 머금...,NaN,3.0,1.0,0.0,프랑스,1.0,0.0,...,1.0,3.0,샤또 르 뷰 셴느,f,0.0,2015,0.0,842,red,그룹 발도뷰


In [70]:
customer_data.to_csv('./data/customer_data.csv', encoding='utf-8-sig')

### questionaire

In [82]:
question1 = {0 : 'black', 1 : 'milk_sweet', 2: 'tea'}
question2 = {0 : 'acid', 1 : 'body'}
question3 = {0 : 'meat', 1 : 'cheese', 2: 'seafood', 3 : 'vegetables', 4 : 'dessert'}
question5 = 0
# 중복 선택 가능
question4 = {0 : 'floral', 1 : 'fruit', 2 : 'citrus', 3 : 'oriental_spice', 4 : 'oriental_leather', 5: 'earth'} 

In [83]:
questions = [question1, question2, question3, question4, question5]
questions 

[{0: 'black', 1: 'milk_sweet', 2: 'tea'},
 {0: 'acid', 1: 'body'},
 {0: 'meat', 1: 'cheese', 2: 'seafood', 3: 'vegetables', 4: 'dessert'},
 {0: 'floral',
  1: 'fruit',
  2: 'citrus',
  3: 'oriental_spice',
  4: 'oriental_leather',
  5: 'earth'},
 0]

In [84]:
# customer's answer
question1_answer = 0
question2_answer = 1
question3_answer = 2
question4_answer = 24
question5_answer = 30000

In [85]:
question1_result = list(question1.values())[question1_answer]
question2_result = list(question2.values())[question2_answer]
question3_result = list(question3.values())[question3_answer]
question4_result = [list(question4.values())[int(i)] if question4_answer >= 10 else list(question4.values())[question4_answer]
                    for i in str(question4_answer)]
question5_result = question5_answer

In [86]:
# split wine data by the answers of the question 1 & 2

if (question1_result, question2_result) == ('black', 'acid'):
    customer_data = customer_data[customer_data['type'] =='a']
elif (question1_result, question2_result) == ('black', 'body'):
    customer_data = customer_data[customer_data['type'] =='b']
elif (question1_result, question2_result) == ('milk_sweet', 'acid'):
    customer_data = customer_data[customer_data['type'] =='c']
elif (question1_result, question2_result) == ('milk_sweet', 'body'):
    customer_data = customer_data[customer_data['type'] =='d']
elif (question1_result, question2_result) == ('tea', 'acid'):
    customer_data = customer_data[customer_data['type'] =='e']
elif (question1_result, question2_result) == ('tea', 'body'):
    customer_data = customer_data[customer_data['type'] =='f']

customer_data.tail(2)

,acidity,alcohol,aroma,aroma_y,body_rate,cheese,citrus,country,dark_aroma,dessert,...,sweetness,tannin_rate,title,type,vegetables,vintage,white,wine_id,wine_type,winery
11998,3.0,14.0,"레드블랜드는 바닐라, 블랙베리의 아로마를 느낄수 있다. 오크터치와 달콤한 과일맛과 ...",NaN,3.0,0.0,0.0,칠레,1.0,0.0,...,1.0,3.0,데나다 셀러 레드 블랜드,b,0.0,2016,0.0,772,red,데나다 셀러
11999,3.0,13.0,어두운 체리색에 선명한 보랏빛 음영. 까베르네 소비뇽에서 느껴지는 우아한 블랙 커런...,NaN,3.0,0.0,0.0,호주,1.0,0.0,...,1.0,3.0,"로즈마운트, 블렌드 까베르네 메를로",b,0.0,2017,0.0,608,red,로즈마운트


In [87]:
# split data by food pairing

if question3_result == 'meat':
    customer_data = customer_data[customer_data['meat'] == 1]
elif question3_result == 'cheese':
    customer_data = customer_data[customer_data['cheese'] == 1]
elif question3_result == 'seafood':
    customer_data = customer_data[customer_data['seafood'] == 1]
elif question3_result == 'vegetables':
    customer_data = customer_data[customer_data['vegatables'] == 1]
elif question3_result == 'dessert':
    customer_data = customer_data[customer_data['dessert'] == 1]
    
customer_data.tail(2)

,acidity,alcohol,aroma,aroma_y,body_rate,cheese,citrus,country,dark_aroma,dessert,...,sweetness,tannin_rate,title,type,vegetables,vintage,white,wine_id,wine_type,winery
11996,3.0,9.0,"노란 볏짚색에 투명한 연두빛을 띠고 메론, 복숭아의 과일향, 토스트와 너트가 느껴지...",NaN,3.0,1.0,0.0,호주,1.0,0.0,...,1.0,1.0,"윈담 이스테이트, 빈 222 샤르도네",b,0.0,2012,1.0,2925,white,윈담 이스테이트
11997,3.0,7.5,"보라빛이 도는 진홍색을 띄고 우수한 베리 과일향, 나뭇잎 민트향, 연한 바닐라 오크...",NaN,3.0,1.0,0.0,호주,1.0,0.0,...,1.0,3.0,"윈담 이스테이트, 빈 888 까베르네 메를로",b,1.0,2012,0.0,2921,red,윈담 이스테이트


In [89]:
# split data by aroma preference

if 'floral' in question4_result:
    customer_data = customer_data[customer_data['floral'] == 1]
elif 'fruity' in question4_result:
    customer_data = customer_data[customer_data['fruity'] == 1]
elif 'citrus' in question4_result:
    customer_data = customer_data[customer_data['citrus'] == 1]
elif 'oriental_spice' in question4_result:
    customer_data = customer_data[customer_data['oriental_spice'] == 1]
elif 'oriental_leather' in question4_result:
    customer_data = customer_data[customer_data['oriental_leather'] == 1]
elif 'earth' in question4_result:
    customer_data = customer_data[customer_data['earth'] == 1]

customer_data.tail(2)

,acidity,alcohol,aroma,aroma_y,body_rate,cheese,citrus,country,dark_aroma,dessert,...,sweetness,tannin_rate,title,type,vegetables,vintage,white,wine_id,wine_type,winery
11969,3.0,9.5,클락스버그 (Clarksburg) 에 위치한 자가소유 포도밭의 포도로만 만든 에스테...,NaN,3.0,0.0,1.0,미국,1.0,0.0,...,1.0,1.0,"랭 트윈스, 샤도네이",b,0.0,2016,1.0,662,white,레인지 트윈즈
11970,3.0,13.5,"레몬, 감귤류의 향과 스파이시한 인동 덩굴 등의 향이 후각을 자극한다. 9개월간 프...",NaN,3.0,1.0,1.0,아르헨티나,1.0,0.0,...,1.0,1.0,"핀카 라스 팔마스, 샤르도네",b,0.0,2015,1.0,1332,white,트라피체


In [90]:
# sort the most popular items
customer_data_counts = customer_data['title'].value_counts()

customer_data = pd.merge(customer_data, customer_data_counts, how='left', left_on ='title',
        right_on =customer_data_counts.index)

customer_data.rename(columns = {'title_y' : 'counts'}, inplace=True)
customer_data.rename(columns = {'title_x' : 'title'}, inplace=True)

customer_data = customer_data.sort_values(by='counts', ascending=False)
customer_data.drop_duplicates(keep='first', inplace=True)
customer_data = customer_data.iloc[:,1:]
customer_data

,acidity,alcohol,aroma,aroma_y,body_rate,cheese,citrus,country,dark_aroma,dessert,...,tannin_rate,title,type,vegetables,vintage,white,wine_id,wine_type,winery,counts
166,3.0,13.5,"깨끗하고 밝은 옐로우 그린 컬러를 가지고 있으며 시트러스, 미네랄 , 사과, 배, ...",NaN,3.0,0.0,1.0,스페인,1.0,0.0,...,1.0,"파고 데 로스 카페자네스, '오 루아르 두 실' 리아스",b,0.0,2015,1.0,410,white,파고 데 로스 카페자네스,12
173,3.0,8.5,"옅은 밀짚색을 띠고, 감귤류의 풍부한 향과 벌꿀향 맛을 가지고 있다. 소비뇽 블랑의...",NaN,3.0,1.0,1.0,미국,1.0,0.0,...,1.0,클렌드넌 패밀리 빈야드 스탈리온 소비뇽 블랑,b,1.0,2014,1.0,639,white,클렌드넌 패밀리 빈야드,10
249,3.0,8.5,"녹색이 살짝 감도는 금빛 반짝이는 노란색을 띠고, 지중해의 각종 허브 향과 프루티한...",NaN,4.0,0.0,1.0,이탈리아,1.0,0.0,...,1.0,칭퀘테레 뻬르골레 스파르세,b,0.0,2013,1.0,1197,white,깐티나 칭퀘테레,9
282,3.0,13.5,"레몬, 감귤류의 향과 스파이시한 인동 덩굴 등의 향이 후각을 자극한다. 9개월간 프...",NaN,3.0,1.0,1.0,아르헨티나,1.0,0.0,...,1.0,"핀카 라스 팔마스, 샤르도네",b,0.0,2015,1.0,1332,white,트라피체,9
76,3.0,13.0,"연한 골드 색상과 레몬, 파인애플, 배, 사과 및 바닐라향, 입안을 감도는 활발한 ...",NaN,3.0,0.0,1.0,미국,1.0,0.0,...,1.0,캘러웨이 샤도네이,b,1.0,2017,1.0,1562,white,캘러웨이 와이너리,9
248,3.0,13.0,"크림같이 부드러운 느낌이 향기롭게 입안에 전해지는 매력적이고 우아한 맛.사과, 열대...",NaN,3.0,1.0,1.0,호주,1.0,0.0,...,1.0,파란드리 샤도네,b,1.0,2003,1.0,2997,white,파란드리,9
117,3.0,8.5,"금빛 반짝이는 노란색을 띠고, 은은히 느껴지는 세이지, 허브 향과 시트러스, 야생 ...",NaN,3.0,0.0,1.0,이탈리아,1.0,0.0,...,1.0,칭퀘테레 코스타 데 캄푸,b,1.0,2013,1.0,1198,white,깐티나 칭퀘테레,8
272,3.0,14.0,"밝은 노란색, 잘 익은 과일과 바닐라, 버터스카치와 같은 달콤한 아로마와 살짝 구운...",NaN,4.0,0.0,1.0,칠레,1.0,0.0,...,1.0,"에라주리즈, 맥스 리제르바 샤르도네",b,0.0,2014,1.0,3047,white,비냐 에라주리즈,8
160,3.0,11.0,생기 있는 그린빛 황금색을 띠고 복숭아와 레몬의 신선한 향이 느껴지며 숙성되지 않았...,NaN,3.0,1.0,1.0,호주,1.0,0.0,...,1.0,피터 르만 아트 시리즈 에덴 밸리 리슬링,b,1.0,2008,1.0,2508,white,피터 르만,8
251,3.0,14.5,"옅은 노란빛을 띠며, 오렌지, 라임, 레몬, 배, 사과, 정향,육두구, 바닐라, 프...",NaN,4.0,0.0,1.0,미국,1.0,0.0,...,1.0,샤플렛 샤르도네,b,0.0,2015,1.0,2155,white,샤플렛,8


In [80]:
# find the best wines without considering price

best_review = pd.DataFrame(columns=['title', 'alcohol', 'sweetness', 'acidity', 'body_rate', 'tannin_rate', 
                      'meat', 'cheese', 'seafood', 'vegetables', 'dessert', 'floral', 'fruit', 'citrus', 'oriental_spice',
                     'oriental_leather', 'earth', 'europe_a', 'europe_b', 'north_america', 'south_america', 
                      'new_world', 'korea','price', 'link'])

data =list(customer_data.iloc[0,:][['title','alcohol', 'sweetness', 'acidity', 'body_rate', 'tannin_rate', 
                      'meat', 'cheese', 'seafood', 'vegetables', 'dessert', 'floral', 'fruit', 'citrus', 'oriental_spice',
                     'oriental_leather', 'earth', 'europe_a', 'europe_b', 'north_america', 'south_america', 
                      'new_world', 'korea', 'price', 'link']])

data = pd.Series(data, index=best_review.columns)

best_review = best_review.append(data, ignore_index=True)
best_review.set_index('title', drop=False, inplace=True)
best_review

,title,alcohol,sweetness,acidity,body_rate,tannin_rate,meat,cheese,seafood,vegetables,...,oriental_leather,earth,europe_a,europe_b,north_america,south_america,new_world,korea,price,link
title,,,,,,,,,,,,,,,,,,,,,
"파고 데 로스 카페자네스, '오 루아르 두 실' 리아스","파고 데 로스 카페자네스, '오 루아르 두 실' 리아스",13.5,1.0,3.0,3.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,70000.0,https://www.wine21.com/13_search/wine_view.htm...


In [81]:
# the best wines considering price(question5_answer)

if best_review['price'][0] > question5_answer:
    customer_data_price = customer_data[customer_data['price'] <= question5_answer]
    customer_data_price = customer_data_price[['title', 'alcohol', 'sweetness', 'acidity', 'body_rate', 'tannin_rate', 
                          'meat', 'cheese', 'seafood', 'vegetables', 'dessert', 'floral', 'fruit', 'citrus', 'oriental_spice',
                         'oriental_leather', 'earth', 'europe_a', 'europe_b', 'north_america', 'south_america', 
                          'new_world', 'korea']]
    best_review.drop(['price', 'link'], axis=1, inplace=True)
    best_review = best_review.append(customer_data_price)
    best_review.set_index('title', inplace=True)
    
    # 커피취향, 음식, 아로마, 가격대가 모두 맞고 베스트 상품과 유사도가 가장 높은 와인
    recommend_wine = cosine_similarity(best_review, best_review)
    recommend_wine_df = pd.DataFrame(data=recommend_wine, index = best_review.index, columns=best_review.index)
    recommend_wine_name = recommend_wine_df.iloc[0,:].sort_values(ascending=False)[1:2]
    for i in range(len(customer_data)):
        if customer_data['title'].iloc[i] == recommend_wine_name.index:
            print(customer_data['link'].iloc[i])
            break
    
else:
    recommend_wine_df = best_review
    print(recommend_wine_df['link'])

https://www.wine21.com/13_search/wine_view.html?Idx=150507
